In [ ]:
#Install dependent packages¶
!pip install gensim
!pip install PyLDAvis
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install langid

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 16.4MB/s 
  Created wheel for PyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=b1384b915f26bdfe27b5014fb5981ceec16d58b0df5042af05eaa63b295e3077
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=40116e59b4769e5ee0a202bac8131b43722d93c088d02f039d43154de3ef3afe
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built PyLDAvis funcy
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 1.9MB 2.8MB/s 
  Created wheel for langid: filename=langid-1.1.6-cp36-none-any.whl size=1941190 sha256=090f3e62715ebff4a7ed88711795f84864c45fe2417181864c2672fb9a24ca0e
  Stored in directory: /r

Model Implementation

Loading data

In [ ]:
import langid
import nltk
import pandas as pd
import re
import glob
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from string import digits

def filter_lang(lang, documents):
    doclang = [  langid.classify(doc) for doc in documents ]
    return [documents[k] for k in range(len(documents)) if doclang[k][0] == lang]



df = pd.read_csv('/content/tweetAOC.csv')
frame=df['tweet']
tweetslist=frame.values.tolist()
tweetslist[:1]
first=filter_lang('en', tweetslist)
first = [re.sub(r"(?:\@|https?\://)\S+", "", doc)
                for doc in first ]


In [ ]:
first

['Guess who’s repeal of the Faircloth Amendment just passed the House! 😌 \n\nFaircloth has blocked construction of new public housing in the United States for 20+ YEARS. Repeal is key to tackling our housing crisis.\n\nTHANK YOU to the advocates who‘ve worked so hard to get here. pic.twitter.com/JkcJvNBr72',
 'The  meme team is really something else 😂 \xa0…',
 'Thank you so much, and thanks for being open to new perspectives. Grateful to have earned your support 💜',
 'Please tell your mom I say thank you and she’s welcome to our team anytime! 💜☺️',
 '🤣 when you don’t realize you’re debating a New Yorker ( \xa0…',
 'One of the ways the GOP articulates their racism today is their idea that some citizens are “real” people and others aren’t.\n\nThis is the argument they make against DC statehood: that this largely Black community doesn’t qualify as “real people.”\n\nSo, who is a “real” person? 🤔 \xa0…',
 'How’s that “anti-AOC” advertising going, Oklahoma GOP?\n\nOh, it ended with voters su

Data Cleaning



Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [ ]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(first))

print(data_words[:1][0][:30])

['guess', 'who', 'repeal', 'of', 'the', 'faircloth', 'amendment', 'just', 'passed', 'the', 'house', 'faircloth', 'has', 'blocked', 'construction', 'of', 'new', 'public', 'housing', 'in', 'the', 'united', 'states', 'for', 'years', 'repeal', 'is', 'key', 'to', 'tackling']


Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.


Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [ ]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['guess', 'amendment', 'pass', 'block', 'construction', 'new', 'public', 'housing', 'year', 'repeal', 'key', 'tackling', 'housing', 'crisis', 'thank', 'advocate', 'work', 'hard', 'com', 'jkcjvnbr']


Data transformation: Corpus and Dictionary

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


Building the base topic model

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

View the topics in LDA model

The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics()

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"tonight" + 0.014*"crisis" + 0.013*"make" + 0.012*"save" + '
  '0.011*"government" + 0.011*"try" + 0.011*"future" + 0.011*"election" + '
  '0.010*"public" + 0.010*"stop"'),
 (1,
  '0.016*"fight" + 0.014*"climate" + 0.012*"take" + 0.011*"justice" + '
  '0.010*"agenda" + 0.009*"war" + 0.009*"leader" + 0.009*"people" + '
  '0.009*"run" + 0.008*"violence"'),
 (2,
  '0.020*"get" + 0.019*"people" + 0.017*"work" + 0.015*"country" + '
  '0.014*"would" + 0.013*"many" + 0.012*"power" + 0.011*"time" + 0.011*"food" '
  '+ 0.010*"family"'),
 (3,
  '0.016*"go" + 0.015*"system" + 0.015*"release" + 0.015*"look" + '
  '0.014*"public" + 0.013*"pay" + 0.013*"come" + 0.011*"know" + 0.010*"cut" + '
  '0.009*"healthcare"'),
 (4,
  '0.021*"people" + 0.015*"dollar" + 0.013*"include" + 0.012*"child" + '
  '0.012*"fight" + 0.011*"care" + 0.011*"pay" + 0.011*"society" + 0.010*"call" '
  '+ 0.009*"many"'),
 (5,
  '0.026*"mean" + 0.017*"time" + 0.014*"together" + 0.014*"vote" + '
  '0.013*"people" + 

Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3544880477267496


In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)

  0%|          | 0/270 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
100%|██████████| 270/270 [36:25<00:00,  8.20s/it]

Final Model Training

Based on external evaluation (Code to be added from Excel based analysis), train the final model

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.003*"word" + 0.002*"good" + 0.002*"hurt" + 0.002*"climate" + '
  '0.002*"leadership" + 0.002*"update" + 0.002*"wild" + 0.002*"hold" + '
  '0.002*"leader" + 0.002*"planet"'),
 (1,
  '0.003*"impeachment" + 0.003*"engage" + 0.002*"transcript" + 0.002*"care" + '
  '0.002*"chairman" + 0.002*"antisemitism" + 0.002*"atrocious" + 0.002*"sure" '
  '+ 0.002*"outright" + 0.002*"understand"'),
 (2,
  '0.014*"people" + 0.008*"get" + 0.007*"work" + 0.006*"public" + 0.006*"year" '
  '+ 0.006*"many" + 0.006*"time" + 0.005*"go" + 0.005*"need" + 0.005*"know"'),
 (3,
  '0.002*"thank" + 0.002*"mention" + 0.002*"bit" + 0.002*"alright" + '
  '0.002*"quite" + 0.002*"upside" + 0.002*"downside" + 0.002*"maybe" + '
  '0.002*"check" + 0.001*"shot"'),
 (4,
  '0.002*"taxi" + 0.002*"president" + 0.002*"immediately" + 0.001*"compare" + '
  '0.001*"city" + 0.001*"resign" + 0.001*"partisanship" + 0.001*"amateur" + '
  '0.001*"react" + 0.001*"trump"'),
 (5,
  '0.009*"enough" + 0.004*"human" + 0.002*"parent" +

In [ ]:
#@title
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.083900 -0.014763       1        1  39.747799
7      0.011911  0.056626       2        1  15.969038
2     -0.006248 -0.004155       3        1  12.065071
1     -0.009500 -0.017000       4        1   9.942390
4     -0.015994 -0.005723       5        1   8.537560
3     -0.014666 -0.008482       6        1   6.043214
5     -0.022931 -0.003224       7        1   4.944599
6     -0.026472 -0.003278       8        1   2.750331, topic_info=        Term       Freq      Total Category  logprob  loglift
390  congrat  18.000000  18.000000  Default  30.0000  30.0000
24        go  40.000000  40.000000  Default  29.0000  29.0000
14       get  23.000000  23.000000  Default  28.0000  28.0000
38      know  29.000000  29.000000  Default  27.0000  27.0000
40       let  22.000000  22.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
185     time   0.377343  16.895563   Topic8  -5.8592  -0.2082
382    young   0.315258  10.128079   Topic8  -6.0390   0.1238
335     hold   0.220180   4.160306   Topic8  -6.3979   0.6546
578      bad   0.209369   3.197246   Topic8  -6.4482   0.8675
756    whole   0.204751   5.295065   Topic8  -6.4705   0.3407

[440 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
8         1  0.364347           aka
1230      1  0.411057  allstardraft
117       1  0.361334         alone
117       3  0.361334         alone
507       1  0.123505       already
...     ...       ...           ...
64        2  0.045129          year
64        6  0.045129          year
382       1  0.592412         young
382       3  0.098735         young
382       4  0.197471         young

[772 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 3, 2, 5, 4, 6, 7])